In [2]:
import numpy as np
import pandas as pd
import pickle

from sklearn.feature_extraction.text import CountVectorizer 
# from scipy.linalg import svd

import scipy.sparse
from sparsesvd import sparsesvd
from scipy.linalg import svd

In [3]:
# assumes transferred files from local to this aws machine via:
# scp -i ~/.ssh/aws_key_sumac.pem data/* ubuntu@54.67.117.7:github_rec/

with open("df.pickle", "rb") as input_file:
    df = pickle.load(input_file)
    
#df.head()

with open("df4.pickle", "rb") as input_file: 
    dfstacked = pickle.load(input_file)
    
# dfstacked.head()

with open("stacked_zeros_df.pickle", "rb") as input_file: 
    stacked_zeros_df = pickle.load(input_file)
    
stacked_zeros_df.head()
# /Users/sumac/projects/metis/github_recommender/data/df.pickle - 3 col
# /Users/sumac/projects/metis/github_recommender/data/df4.pickle - stacked format

,following,rating,username
0,josh,1.0,nishowsan
1,benschwarz,1.0,nishowsan
2,andrew,1.0,nishowsan
3,tmm1,1.0,nishowsan
4,mitsuhiko,1.0,nishowsan


In [4]:
stacked_zeros_df.rename(columns={'rating':'all_ratings'}, inplace=True)

In [5]:
stacked_zeros_df = stacked_zeros_df[['username', 'following', 'all_ratings']]

# using surprise to get recs

In [8]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

stacked_zeros_df.head()

reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
#data = Dataset.load_from_df(stacked_zeros_df[['username', 'following', 'rating']], reader)
data2 = Dataset.load_from_df(stacked_zeros_df[['username', 'following', 'all_ratings']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
model1 = cross_validate(NormalPredictor(), data2, cv=2)

In [9]:
# np = NormalPredictor()

In [51]:
# np.fit(data2)

In [50]:
# # can we extract the predictions?
# # code from surprise website

# uid = str(196)  # raw user id (as in the ratings df). They are **strings**!
# iid = str(302)  # raw item id (as in the ratings df). They are **strings**!

# # get a prediction for specific users and items
# pred = algo.predict(uid, iid, r_ui=4, verbose=True)

In [ ]:
# testing

In [10]:
from surprise import KNNBasic
from surprise import Dataset


# Retrieve the trainset.
trainset = data2.build_full_trainset()

# Build an algorithm and train it
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [47]:
# # test known True following pair
# uid = 'nishowsan'  # raw user id - string
# iid = 'andrew'  # raw item id - string

# # get a prediction for specific user and items
# pred = algo.predict(uid, iid, r_ui=4, verbose = True)

user: nishowsan  item: andrew     r_ui = 4.00   est = 1.00   {'actual_k': 40, 'was_impossible': False}


In [49]:
# pred.est

1

In [11]:
with open("df4.pickle", "rb") as input_file: 
    df_stacked = pickle.load(input_file)
    
df_stacked.head()

,username,rating,following
0,donnemartin,1,mitsuhiko
1,donnemartin,1,kenneth-reitz
2,donnemartin,1,wesm
3,donnemartin,1,jakevdp
4,mitsuhiko,1,brosner


### get predictions

In [12]:
df.sample(7)

,username,following,rating
30133,johbo,netme qknight PythonicNinja ticosax globin vcu...,1
19656,python-bugzilla,,1
22661,alex2,mtarbit ask toastdriven caolan dtt101 Jc2k jac...,1
9528,orhanobut,JakeWharton yekmer tasomaniac keklikhasan oguz...,1
3542,neil-tan,,1
28129,kapsiry,,1
334,aziz,defunkt technoweenie drnic peterc ryanb rtomay...,1


In [13]:
def get_not_following(username):

    following = set(df.loc[df['username'] == username, 'following'])  # get list username _is_ following
    
    all_follows = set(df_stacked['username'].unique() + dfstacked['following']) # get list everyone has followed, all poss.

    not_following = list(all_follows - following) # difference is list of people username is not following    
        
    return not_following

In [14]:
recs = {} # collect to export for flask app

In [ ]:
# need to loop over each user in original df, run prediction on each user

target_user = 'vassim'

# need func to get list of users in "following" column, minus those entries for 

not_following_list = get_not_following(target_user) # [ALL THE USERS target_user IS NOT FOLLOWING] 

estimated_scores = []

for user in not_following_list:

    current_score = algo.predict(target_user, user).est

    estimated_scores.append((current_score, user))

#print(estimated_scores)
sorted_scores = sorted(estimated_scores, reverse = True, key = lambda x: x[0])

recs[target_user] = sorted_scores[0:5]

In [ ]:
print(recs)

### export predictions table for use in flask app for quick demo solution

In [73]:
import json
 
json = json.dumps(recs)
f = open("recs.json","w")
f.write(json)
f.close()

In [ ]:
# remember to pull down this file and recs.json to local

# unused code below

### create sparse/dense matrices for use with non-surprise libs

In [14]:
stacked_zeros_df.head()

,following,rating,username
0,josh,1.0,nishowsan
1,benschwarz,1.0,nishowsan
2,andrew,1.0,nishowsan
3,tmm1,1.0,nishowsan
4,mitsuhiko,1.0,nishowsan


In [3]:
df = df.drop('rating', axis = 1)

In [4]:
cv = CountVectorizer()

In [5]:
cv1 = cv.fit_transform(df.following) # creates dense matrix

In [6]:
type(cv1)

scipy.sparse.csr.csr_matrix

In [12]:
cv2 = cv1.tocsc()

In [13]:
print(cv2)

  (6966, 0)	1
  (6920, 1)	1
  (8709, 2)	1
  (45105, 3)	1
  (23822, 4)	1
  (6603, 5)	1
  (9021, 5)	1
  (5416, 6)	1
  (1948, 7)	1
  (6901, 7)	1
  (13810, 7)	1
  (37018, 7)	1
  (48907, 7)	1
  (45386, 8)	1
  (46137, 9)	1
  (48401, 10)	1
  (2163, 11)	1
  (68, 12)	1
  (17032, 13)	1
  (40101, 14)	1
  (32251, 15)	1
  (26274, 16)	1
  (35478, 17)	1
  (10450, 18)	1
  (31307, 18)	1
  :	:
  (45025, 175775)	1
  (45645, 175775)	1
  (45743, 175775)	1
  (47977, 175775)	1
  (5794, 175776)	1
  (37297, 175777)	1
  (25464, 175778)	1
  (44837, 175779)	1
  (23125, 175780)	1
  (38759, 175781)	1
  (42594, 175782)	1
  (29728, 175783)	1
  (42899, 175784)	1
  (5208, 175785)	1
  (31713, 175785)	1
  (10944, 175786)	1
  (21980, 175787)	1
  (39307, 175787)	1
  (18348, 175788)	1
  (32787, 175788)	1
  (44587, 175788)	1
  (16434, 175789)	1
  (20052, 175790)	1
  (42461, 175790)	1
  (45882, 175791)	1


## try with sparsesvd?

In [ ]:
# U, Sigma, VT = svd(cv2)

In [17]:
ut, s, vt = sparsesvd(cv2, 100) # https://pypi.org/project/sparsesvd/

In [39]:
vt.shape

(100, 175792)

In [50]:
def get_recommends(itemID, vt, num_recom=4):
    recs = []
    for item in range(vt.T.shape[0]):
        if item != itemID:
            recs.append([item,np.dot(vt.T[itemID],vt.T[item])])
    #print(recs[:4])
    #print(len(recs))
    final_rec = [i[0] for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

print(get_recommends(0,vt,1))

[109919]


In [41]:
vt.T[0]

array([ 2.13581707e-06, -2.62145678e-06, -6.60407669e-07, -5.73848740e-08,
        5.15191641e-06,  7.55844065e-06, -3.10182000e-06, -1.52120974e-05,
        1.81528372e-05,  1.82933947e-05, -3.97383912e-06,  9.94486488e-07,
       -7.03583927e-06,  8.03047356e-06,  2.28077663e-05,  5.70432093e-06,
       -1.16204214e-05, -2.50122717e-05,  1.67223970e-05,  1.57086305e-05,
       -2.61964427e-05, -1.29284410e-05, -1.02520769e-05, -1.31802649e-05,
        8.70263187e-06,  1.54513950e-05, -5.72642714e-05, -2.68605490e-05,
       -1.30359171e-05, -1.79817007e-05,  2.12396259e-05,  5.68859637e-06,
        2.37622986e-05, -1.57213702e-05, -5.11272912e-05, -3.21831649e-05,
        4.29779352e-05,  5.06508499e-05, -2.11629070e-05,  4.77075291e-05,
        6.26479932e-05, -2.36916005e-05, -1.53229047e-05,  6.29408853e-05,
       -3.53995648e-05,  1.58488552e-05,  4.54988493e-05, -1.31862795e-05,
       -1.93202359e-05, -2.21882812e-05, -8.10110230e-06,  4.26991634e-05,
       -2.74455158e-05, -

In [34]:
df.shape

(49189, 2)

In [ ]:
def get_recommends_user(userID, U, df):
    userrecs = []
    for user in range(U.shape[0]):
        if user!= userID:
            userrecs.append([user,np.dot(U[userID],U[user])])
    final_rec = [i[0] for i in sorted(userrecs,key=lambda x: x[1],reverse=True)]
    comp_user = final_rec[0]
    print("User #%s's most similar user is User #%s "% (userID, comp_user))
    rec_likes = df.iloc[comp_user]
    current = df.iloc[userID]
    recs = []
    for i,item in enumerate(current):
        if item != rec_likes[i] and rec_likes[i]!=0:
            recs.append(i)
    return recs

user_to_rec = 0

print("Items for User %s to check out: "% user_to_rec, get_recommends_user(user_to_rec,ut,df))

In [ ]:
df.iloc([[0]])

## try with knn surprise method?

In [ ]:
# code from https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-k-nearest-neighbors-of-a-user-or-item

#import io  # needed because of weird encoding of u.item file
from surprise import KNNBaseline
from surprise import Dataset
#from surprise import get_dataset_dir


def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

#     file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
#     rid_to_name = {}
#     name_to_rid = {}

    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

###
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(dfs[['username', 'following', 'rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)
###



# First, train the algortihm to compute the similarities between items
# data = Dataset.load_builtin('ml-100k')
# trainset = data.build_full_trainset()


sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

print()
print('The 10 nearest neighbors of Toy Story are:')
for movie in toy_story_neighbors:
    print(movie)

In [ ]:
# predict rating eventually

uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)